# Transforming Complex Data Types in Spark SQL
In this notebook we're going to go through some data transformation examples using Spark SQL. Spark SQL supports many built-in transformation functions in the module pyspark.sql.functions therefore we will start off by importing that.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName('complex').getOrCreate()

### Method for creating df out of json data

In [3]:
def json_to_DF(json, schema = None):
    reader = spark.read
    if schema:
        reader.schema(schema)
    return reader.json(spark.sparkContext.parallelize([json]))

## Selecting from nested columns - Dots (".") can be used to access nested columns for structs and maps.

In [10]:
schema = StructType().add("a", StructType().add("b", IntegerType()))

json_data = """
{
  "a": {
     "b": 1
  }
}
"""

json_df = json_to_DF(json_data, schema)
json_df.printSchema()
json_df.show()
json_df.select("a.b").show()

root
 |-- a: struct (nullable = true)
 |    |-- b: integer (nullable = true)

+---+
|  a|
+---+
|{1}|
+---+

+---+
|  b|
+---+
|  1|
+---+



## Using a map type

In [13]:
schema = StructType().add("a", MapType(StringType(), IntegerType()))
json_map_df = json_to_DF(json_data, schema)
json_map_df.printSchema()
json_map_df.show()
json_map_df.select("a.b").show()

root
 |-- a: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = true)

+--------+
|       a|
+--------+
|{b -> 1}|
+--------+

+---+
|  b|
+---+
|  1|
+---+



## Flattening structs - A star ("*") can be used to select all of the subfields in a struct.

In [20]:
# Map schema doesnt seem to support .*
schema = StructType().add("a", MapType(StringType(), IntegerType()))

df = json_to_DF("""
{
  "a": {
     "b": 1,
     "c": 2
  }
}
""")

df.select("a.*").show()

+---+---+
|  b|  c|
+---+---+
|  1|  2|
+---+---+



## Nesting columns - The struct() function or just parentheses in SQL can be used to create a new struct.

In [22]:
df = json_to_DF("""
{
  "a": 1,
  "b": 2,
  "c": 3
}
""")
df.show()
df.select(struct(col("a").alias("y")).alias("x")).show()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
+---+---+---+

+---+
|  x|
+---+
|{1}|
+---+



## Nesting all columns - The star ("*") can also be used to include all columns in a nested struct.

In [25]:
df.select(struct("*").alias("all")).show()

+---------+
|      all|
+---------+
|{1, 2, 3}|
+---------+



## Selecting a single array or map element - getItem() or square brackets (i.e. [ ]) can be used to select a single element out of an array or a map.

In [27]:
df = json_to_DF("""
{
  "a": [1, 2]
}
""")

df.show()
df.select(col("a").getItem(0).alias('x')).show()

+------+
|     a|
+------+
|[1, 2]|
+------+

+---+
|  x|
+---+
|  1|
+---+



In [28]:
# Using a map
schema = StructType().add("a", MapType(StringType(), IntegerType()))
 
df = json_to_DF("""
{
  "a": {
    "b": 1
  }
}
""", schema)

df.select(col("a").getItem("b").alias("x")).show()

+---+
|  x|
+---+
|  1|
+---+



## Creating a row for each array or map element - explode() can be used to create a new row for each element in an array or each key-value pair. This is similar to LATERAL VIEW EXPLODE in HiveQL.

In [29]:
df = json_to_DF("""
{
    "a":[1,2]
}
""")

df.select(explode("a").alias("exploded")).show()

+--------+
|exploded|
+--------+
|       1|
|       2|
+--------+



In [31]:
# Using a map
schema = StructType().add("a", MapType(StringType(), IntegerType()))
 
df = json_to_DF("""
{
  "a": {
    "b": 1,
    "c": 2
  }
}
""", schema)

df.show()
df.select(explode("a").alias("x","y")).show()

+----------------+
|               a|
+----------------+
|{b -> 1, c -> 2}|
+----------------+

+---+---+
|  x|  y|
+---+---+
|  b|  1|
|  c|  2|
+---+---+



## Collecting multiple rows into an array - collect_list() and collect_set() can be used to aggregate items into an array.

In [32]:
df = json_to_DF("""
[{ "x": 1 }, { "x": 2 }]
""")
 
df.select(collect_list("x").alias("x")).show()

+------+
|     x|
+------+
|[1, 2]|
+------+



In [41]:
# using an aggregation
df = json_to_DF("""
[{ "x": 1, "y": "a" }, { "x": 2, "y": "b" }]
""")

df.show()

df.select("x").show()

# to explode both the column we have to use aggregation

df.groupBy("y").agg(collect_list("x").alias("x")).show()

+---+---+
|  x|  y|
+---+---+
|  1|  a|
|  2|  b|
+---+---+

+---+
|  x|
+---+
|  1|
|  2|
+---+

+---+---+
|  y|  x|
+---+---+
|  b|[2]|
|  a|[1]|
+---+---+



## Selecting one field from each item in an array - when you use dot notation on an array we return a new array where that field has been selected from each array element.

In [43]:
df = json_to_DF("""
{
  "a": [
    {"b": 1},
    {"b": 2}
  ]
}
""")
 
df.show()

df.select("a.b").show()

+----------+
|         a|
+----------+
|[{1}, {2}]|
+----------+

+------+
|     b|
+------+
|[1, 2]|
+------+



## Convert a group of columns to json - to_json() can be used to turn structs into json strings. This method is particularly useful when you would like to re-encode multiple columns into a single one when writing data out to Kafka. This method is not presently available in SQL.

In [44]:
df = json_to_DF("""
{
  "a": {
    "b": 1
  }
}
""")

df.select(to_json("a").alias("c")).show()

+-------+
|      c|
+-------+
|{"b":1}|
+-------+



## Parse a column containing json - from_json() can be used to turn a string column with json data into a struct. Then you may flatten the struct as described above to have individual columns. This method is not presently available in SQL. This method is available since Spark 2.1

In [45]:
df = json_to_DF("""
{
  "a": "{\\"b\\":1}"
}
""")
 
schema = StructType().add("b", IntegerType())
df.select(from_json("a", schema).alias("c")).show()

+---+
|  c|
+---+
|{1}|
+---+



## Sometimes you may want to leave a part of the JSON string still as JSON to avoid too much complexity in your schema.

In [48]:
df = json_to_DF("""
{
  "a": "{\\"b\\":{\\"x\\":1,\\"y\\":{\\"z\\":2}}}"
}
""")
 
schema = StructType().add("b", StructType().add("x", IntegerType())
                            .add("y", StringType()))

df.show(truncate=False)
df.select(from_json("a", schema).alias("c")).show()

+-------------------------+
|a                        |
+-------------------------+
|{"b":{"x":1,"y":{"z":2}}}|
+-------------------------+

+--------------+
|             c|
+--------------+
|{{1, {"z":2}}}|
+--------------+



## Parse a set of fields from a column containing json - json_tuple() can be used to extract a fields available in a string column with json data.

In [50]:
df = json_to_DF("""
{
  "a": "{\\"b\\":1}"
}
""")

df.show()
 
df.select(json_tuple("a", "b").alias("c")).show()

+-------+
|      a|
+-------+
|{"b":1}|
+-------+

+---+
|  c|
+---+
|  1|
+---+



## Parse a well formed string column - regexp_extract() can be used to parse strings using regular expressions.

In [51]:
df = json_to_DF("""
[{ "a": "x: 1" }, { "a": "y: 2" }]
""")

df.show()

df.select(regexp_extract("a", "([a-z]):", 1).alias("c")).show()

+----+
|   a|
+----+
|x: 1|
|y: 2|
+----+

+---+
|  c|
+---+
|  x|
|  y|
+---+

